# Limpieza de los datos
En esta sección vamos clasificar cada columna del set de datos por un tipo específico e.g., categóricos, numéricos, cadenas, etc. y decidir que hacer con los posibles atributos que no respeten dicha clasificación.

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', 25)

In [3]:
df = pd.read_csv('../../fiuba-trocafone-tp1-final-set/events.csv', low_memory=False)

## Clasificación de los datos
Debido a la descripción proporcionada en el enunciado del trabajo práctico espero que los siguientes atributos sean __categoricos__:
- [event](#event)
- [person](#person)
- [url](#url)
- [sku](#sku)
- [model](#model)
- [condition](#condition)
- [storage](#storage)
- [color](#color)
- [staticpage](#staticpage)
- [campaign_source](#campaign_source)
- [search_engine](#search_engine)
- [channel](#channel)
- [new_vs_returning](#new_vs_returning)
- [city](#city)
- [region](#region)
- [country](#country)
- [device_type](#device_type)
- [operating_system_version](#operating_system_version)
- [browser_version](#browser_version)

Para comprobarlo transformo los datos en categóricos y observo qué categorías se generan.

Por otra parte espero que los siguiente atributos no lo sean:
- [screen_resolution](#screen_resolution)


In [4]:
atributos_categoricos = ['event', 'person', 'url', 'sku', 'model', 'condition', 'storage', 'color', 'staticpage',
                        'campaign_source', 'search_engine', 'channel', 'new_vs_returning', 'city', 'region', 'country',
                        'device_type', 'operating_system_version', 'browser_version']
for atributo in atributos_categoricos:
    df[atributo] = df[atributo].astype('category')

Ahora podemos revisar las categorías de cada atributo:

<h4 id="event">event</h4>

In [5]:
df['event'].cat.categories

Index(['ad campaign hit', 'brand listing', 'checkout', 'conversion',
       'generic listing', 'lead', 'search engine hit', 'searched products',
       'staticpage', 'viewed product', 'visited site'],
      dtype='object')

In [6]:
df[pd.isnull(df['event'])].size

0

En este caso vemos que los eventos coinciden con los descritos en el enunciado y no tenemos ningún nulo.

<h4 id="person">person</h4>

In [7]:
df['person'].cat.categories

Index(['0004b0a2', '0006a21a', '000a54b2', '00184bf9', '0019c395', '001bb7eb',
       '001f1653', '00204059', '0020f73c', '0024a82b',
       ...
       'ffef9da6', 'fff0e00c', 'fff225f8', 'fff229f7', 'fff568f7', 'fff60213',
       'fff99b85', 'ffff8106', 'ffffa8d1', 'ffffac8a'],
      dtype='object', length=27624)

Notamos que las categorias de los usuarios se parecen mucho a números en hexadecimal. Podriamos pasarlo a binario o a decimal pero no ganariamos nada dado que no resulta util calcular promedios, varianzas, etc. sobre un identificador. Por otro lado puede resultar interesante confirmar si los datos de esta columna se corresponden a números en hexadecimal simplemente para confirmar la integridad de los datos e identificar anomalías.

In [8]:
person_in_decimal = df['person'].apply(lambda x: int(x, 16))
pd.to_numeric(person_in_decimal);

Vemos que no hubo problemas en la conversion.

In [9]:
df[pd.isnull(df['person'])].size

0

A su vez vemos que no hay ningún valor nulo.

<h4 id="url">url</h4>

In [10]:
df['url'].cat.categories

Index(['/', '/clube-trocafone', '/comprar/apple/iphone-6s-plus',
       '/comprar/apple/iphone-7-plus', '/comprar/asus/zenfone-2',
       '/comprar/asus/zenfone-2-deluxe', '/comprar/asus/zenfone-2-laser',
       '/comprar/asus/zenfone-3-max-16gb', '/comprar/asus/zenfone-3-max-32gb',
       '/comprar/asus/zenfone-5',
       ...
       '/vender/lg', '/vender/lg/lg-l80-dual', '/vender/motorola',
       '/vender/motorola/moto-e-2a-geracao-4g-dual',
       '/vender/motorola/moto-g5-plus', '/vender/motorola/moto-g5s-plus',
       '/vender/motorola/moto-z2-force', '/vender/motorola/moto-z2-play',
       '/vender/samsung/galaxy-s5', '/vender/samsung/galaxy-s8-plus'],
      dtype='object', length=227)

In [11]:
df[pd.isnull(df['url'])].shape

(928532, 23)

En este caso sí hay varios nulos.

<h4 id="sku">sku</h4>

In [12]:
df['sku'].cat.categories

Index(['10000', '10000.0', '10001', '10001.0', '10002', '10002.0', '10014',
       '10014.0', '10015', '10015.0',
       ...
       '9973.0', '9974', '9974.0', '9986', '9986.0', '9987', '9987.0', '9988',
       '9988.0', 'undefined'],
      dtype='object', length=3574)

En este caso vemos que hay varias categorias numéricas (aunque por ahora no los convertimos) y una 'undefined'. Primero vamos a ver que hay dentro de 'undefined' y luego trataremos de convertirlas a datos numéricos.

In [13]:
undefined_skus = df[df['sku'] == 'undefined']

Se ve que son solo dos y que todos los atributos son valores nulos exceptuando el de persons, así que sospechamos que son eventos que no aportan informción realmente. Para confirmarlo podemos buscar otros eventos realizados por los mismos usuarios y ver si hay otro en un timestamp cercano.

In [14]:
df[df['person'].isin(undefined_skus['person'])]

,timestamp,event,person,url,sku,model,condition,storage,color,skus,search_term,staticpage,campaign_source,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
368096,2018-05-22 13:53:02,visited site,602b3649,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Direct,New,Unknown,Unknown,Brazil,Computer,1366x768,Windows 7,Chrome 66.0
368097,2018-05-22 13:53:14,checkout,602b3649,NaN,undefined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
387149,2018-05-22 13:44:53,checkout,655402b0,NaN,undefined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
387150,2018-05-22 13:44:53,visited site,655402b0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Direct,New,Unknown,Unknown,Brazil,Computer,1366x768,Windows 7,Chrome 66.0


Se ve que en ambos casos existe otro evento de tipo 'visited site' a pocos segundos del primer evento, por lo cual podemos descartarlos sabiendo que existe otro con mas información que representa la visita del usuario.

In [15]:
df.drop(index=undefined_skus.index, inplace=True)

Ahora podemos transformar el 'sku' a un tipo numerico.

In [16]:
df['sku'] = pd.to_numeric(df['sku'])

Y ahora ya nos aseguramos que todos son del mismo tipo podemos volver a convertirlos en categorias.

In [17]:
df['sku'] = df['sku'].astype('category')

<h4 id="model">model</h4>

In [18]:
df['model'].cat.categories

Index(['Asus Live', 'Asus Zenfone 2', 'Asus Zenfone 2 Deluxe',
       'Asus Zenfone 2 Laser', 'Asus Zenfone 3 Max  32 GB',
       'Asus Zenfone 3 Max 16 GB', 'Asus Zenfone 3 Zoom', 'Asus Zenfone 5',
       'Asus Zenfone 6', 'Asus Zenfone Go',
       ...
       'iPhone 6', 'iPhone 6 Plus', 'iPhone 6S', 'iPhone 6S Plus', 'iPhone 7',
       'iPhone 7 Plus', 'iPhone 8', 'iPhone 8 Plus', 'iPhone SE', 'iPhone X'],
      dtype='object', length=202)

<h4 id="condition">condition</h4>

In [19]:
df['condition'].cat.categories

Index(['Bom', 'Bom - Sem Touch ID', 'Excelente', 'Muito Bom', 'Novo'], dtype='object')

Acá notamos que los valores de este atributo están en portugués, por lo que aprovechamos para traducirlos a español y así trabajar más comodamente.

In [20]:
condition_translations = {
    'Bom': 'Bueno',
    'Bom - Sem Touch ID': 'Bueno - Sin Touch ID',
    'Excelente': 'Excelente',
    'Muito Bom': 'Muy bueno',
    'Novo': 'Nuevo'
}

df['condition'].cat.rename_categories(condition_translations, inplace=True)

In [21]:
df['condition'].cat.categories

Index(['Bueno', 'Bueno - Sin Touch ID', 'Excelente', 'Muy bueno', 'Nuevo'], dtype='object')

<h4 id="storage">storage</h4>

In [22]:
df['storage'].cat.categories

Index(['128GB', '16GB', '256GB', '32GB', '4GB', '512MB', '64GB', '8GB'], dtype='object')

Puede resultar buena idea pasar este atributo a valores numéricos dado que la capacidad de almacenamiento es un valor numérico, pero por otro lado sabemos que la capacidad de los smartphones toma solo ciertos valores determinados por lo que resulta conveniente dejarlo como categorías.
Sin embargo sí resultaría muy útil medir todos los datos con la misma unidad y dado que observamos que la mayoría de los datos están en GB decidimos utilizar esta unidad.

In [23]:
storage_translations = {}

for almacenamiento in df['storage'].cat.categories:
    capacidad = float(almacenamiento[:-2])
    storage_translations[almacenamiento] = capacidad
storage_translations['512MB'] = 0.5

df['storage'].cat.rename_categories(storage_translations, inplace=True)

In [24]:
df['storage'].cat.categories

Float64Index([128.0, 16.0, 256.0, 32.0, 4.0, 0.5, 64.0, 8.0], dtype='float64')

A su vez para dejar en claro la unidad en la que estamos trabajando renombraremos la columna a 'storage_gb'.

In [25]:
df.rename(columns={'storage': 'storage_gb'});

<h4 id="color">color</h4>

In [26]:
df['color'].cat.categories

Index(['Amarelo', 'Ametista', 'Azul', 'Azul Escuro', 'Azul Safira',
       'Azul Topázio', 'Bambu', 'Black Piano', 'Branco', 'Branco Azul',
       'Branco Azul Navy', 'Branco Bambu', 'Branco Cabernet', 'Branco Dourado',
       'Branco Framboesa', 'Branco Pink', 'Branco Verde', 'Branco Vermelho',
       'Cabernet', 'Cinza', 'Cinza espacial', 'Cobre', 'Coral', 'Couro Marrom',
       'Couro Navy', 'Couro Vinho', 'Couro Vintage', 'Cromo', 'Dourado',
       'Framboesa', 'Indigo', 'Iuna', 'Olympic Edition', 'Ouro', 'Ouro Rosa',
       'Platinum', 'Prata', 'Prateado', 'Preto', 'Preto Asfalto', 'Preto Azul',
       'Preto Azul Navy', 'Preto Bambu', 'Preto Branco', 'Preto Brilhante',
       'Preto Cabernet', 'Preto Matte', 'Preto Pink', 'Preto Tabaco',
       'Preto Verde', 'Preto Vermelho', 'Rosa', 'Rose', 'Rouge', 'Roxo',
       'Silver', 'Titânio', 'Turquesa', 'Verde', 'Verde Petroleo',
       'Verde Água', 'Vermelho', 'Ônix'],
      dtype='object')

Al igual que con el almacenamiento podemos traducir las categorias a español.

En algunos casos se encontraron colores un poco 'extravagantes' o sin un significado concreto, por lo que se procedió a buscar más información sobre el mismo a partir del modelo de smartphone al que corresponden. Un ejemplo de ello es el color 'Iuna':

In [27]:
df[(df['color'] == 'Iuna')]

,timestamp,event,person,url,sku,model,condition,storage,color,skus,search_term,staticpage,campaign_source,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
1766,2018-05-15 20:51:24,viewed product,009696f0,NaN,3098.0,Motorola Moto X2,Excelente,32.0,Iuna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14154,2018-05-23 13:19:14,viewed product,03b49fa6,NaN,3098.0,Motorola Moto X2,Excelente,32.0,Iuna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19117,2018-03-27 20:06:40,viewed product,05302c46,NaN,3095.0,Motorola Moto X2,Bueno,32.0,Iuna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46471,2018-06-08 12:33:08,viewed product,0d669c41,NaN,3095.0,Motorola Moto X2,Bueno,32.0,Iuna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50007,2018-02-24 00:38:15,viewed product,0e8fc68b,NaN,3098.0,Motorola Moto X2,Excelente,32.0,Iuna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50344,2018-03-21 17:43:25,viewed product,0e8fc68b,NaN,3098.0,Motorola Moto X2,Excelente,32.0,Iuna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50453,2018-04-04 23:07:44,viewed product,0e8fc68b,NaN,3095.0,Motorola Moto X2,Bueno,32.0,Iuna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54130,2018-05-16 03:25:51,viewed product,0f9bb8c5,NaN,3095.0,Motorola Moto X2,Bueno,32.0,Iuna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54131,2018-05-16 03:26:02,viewed product,0f9bb8c5,NaN,8919.0,Motorola Moto X2,Bueno,16.0,Iuna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54132,2018-05-16 03:26:10,viewed product,0f9bb8c5,NaN,3095.0,Motorola Moto X2,Bueno,32.0,Iuna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df[(df['color'] == 'Iuna') & (df['model']!= 'Motorola Moto X2')].size

0

Al buscar el modelo 'Motorola Moto X2' (que es el único que aparece en el set de datos con el color 'Iuna') en www.trocafone.com, se encontró que dicho color pertenecía a una smartphone con un dibujo de madera en la carcaza (https://www.trocafone.com/comprar/motorola/moto-x-2a-geracao#242-motorola-moto-x2-32gb-iuna-bom-15). Por ello se procedió a tracucir este color como 'Madera'.

Algo similar ocurrió con otros colores, como por ejemplo el 'Branco Azul'. En este caso vemos que la traducción literal sería  'Blanco Azul'. Esto pordría significar celeste, o también un smartphone con dos colores. Para comprobarlo buscamos los modelos con este color:

In [29]:
df[df['color'] == 'Branco Azul']['model'].unique()

[Motorola Moto G3 HDTV, Motorola Moto G3 4G]
Categories (2, object): [Motorola Moto G3 HDTV, Motorola Moto G3 4G]

Luego al buscar estos modelos notamos que son blancos en la parte delantera y azules en la trasera. Para reflejar esto de manera más sencilla se utilizó como traducción 'Blanco,Azul', de manera tal que cuando se encuentre un smartphone con una el formato 'color1,color2' se entienda que el color1 es el de la parte delantera y color2 el de la trasera. Si se encuentra el formato 'color', se entenderá que el celuar es de un solo color.

In [30]:
color_translations = {
    'Amarelo': 'Amarillo',
    'Ametista': 'Amatista',
    'Azul': 'Azul',
    'Azul Escuro': 'Azul Oscuro',
    'Azul Safira': 'Azul Zafiro',
    'Azul Topázio': 'Azul Topacio',
    'Bambu': 'Bambu',
    'Black Piano': 'Negro Piano',
    'Branco': 'Blanco',
    'Branco Azul': 'Blanco,Azul',
    'Branco Azul Navy': 'Blanco,Azul Navy',
    'Branco Bambu': 'Blanco,Bambu',
    'Branco Cabernet': 'Blanco,Cabernet',
    'Branco Dourado': 'Blanco,Dorado',
    'Branco Framboesa': 'Blanco,Franbuesa',
    'Branco Pink': 'Blanco,Rosa',
    'Branco Verde': 'Blanco,Verde',
    'Branco Vermelho': 'Blanco,Rojo',
    'Cabernet': 'Cabernet',
    'Cinza': 'Gris',
    'Cinza espacial': 'Gris Espacial',
    'Cobre': 'Cobre',
    'Coral': 'Coral',
    'Couro Marrom': 'Cuero Marrón',
    'Couro Navy': 'Cuero Navy',
    'Couro Vinho': 'Cuero Vino',
    'Couro Vintage': 'Cuero Vintage',
    'Cromo': 'Cromo',
    'Dourado': 'Dorado',
    'Framboesa': 'Frambuesa',
    'Indigo': 'Indigo',
    'Iuna': 'Madera',
    'Olympic Edition': 'Edicion Olimpica',
    'Ouro': 'Oro',
    'Ouro Rosa': 'Oro Rosa',
    'Platinum': 'Platino',
    'Prata': 'Plata',
    'Prateado': 'Plateado',
    'Preto': 'Negro',
    'Preto Asfalto': 'Negro Asfalto',
    'Preto Azul': 'Negro,Azul',
    'Preto Azul Navy': 'Negro,Azul Navy',
    'Preto Bambu': 'Negro,Bambu',
    'Preto Branco': 'Negro,Blanco',
    'Preto Brilhante': 'Negro Brillante',
    'Preto Cabernet': 'Negro,Cabernet',
    'Preto Matte': 'Negro Mate',
    'Preto Pink': 'Negro,Rosa',
    'Preto Tabaco': 'Negro,Tabaco',
    'Preto Verde': 'Negro,Verde',
    'Preto Vermelho': 'Negro,Rojo',
    'Rosa': 'Rosa',
#     'Rose': 'Rosa',
    'Rouge': 'Rouge',
    'Roxo': 'Purpura',
#     'Silver': 'Plata',
    'Titânio': 'Titanio',
    'Turquesa': 'Turquesa',
    'Verde': 'Verde',
    'Verde Petroleo': 'Verde Petroleo',
    'Verde Água': 'Verde Agua',
    'Vermelho': 'Rojo',
    'Ônix': 'Onice'
}

df['color'] = df['color'].cat.rename_categories(color_translations)

Como no podemos renombrar dos categorías al mismo nombre, queda cambiar los valores de 'Rose' y 'Silver'.

In [31]:
df['color'].loc[df['color'] == 'Rose'] = 'Rosa'
df['color'].loc[df['color'] == 'Silver'] = 'Plata'
df['color'].cat.remove_unused_categories(inplace=True)

<h4 id="staticpage">staticpage</h4>

In [32]:
df['staticpage'].cat.categories

Index(['AboutUs', 'Conditions', 'CustomerService', 'FaqEcommerce',
       'PrivacyEcommerce', 'Quiosks', 'TermsAndConditionsEcommerce',
       'TermsAndConditionsReturnEcommerce', 'black_friday', 'club-trocafone',
       'galaxy-s8', 'how-to-buy', 'how-to-sell', 'trust-trocafone'],
      dtype='object')

<h4 id="campaign_source">campaign_source</h4>

In [33]:
df['campaign_source'].cat.categories

Index(['Facebook', 'FacebookAds', 'FacebookSocial', 'Google Social',
       'MARKETING SOCIAL', 'afiliado', 'afilio', 'bing', 'blog', 'buscape',
       'criteo', 'datacrush', 'emblue', 'google', 'indexa', 'manifest',
       'mercadopago', 'onsite', 'rakuten', 'rtbhouse', 'socialmedia', 'voxus',
       'yotpo', 'zanox'],
      dtype='object')

<h4 id="search_engine">search_engine</h4>

In [34]:
df['search_engine'].cat.categories

Index(['Ask', 'Bing', 'Google', 'Yahoo'], dtype='object')

<h4 id="channel">channel</h4>

In [35]:
df['channel'].cat.categories

Index(['Direct', 'Email', 'Organic', 'Paid', 'Referral', 'Social', 'Unknown'], dtype='object')

Acá notamos que para los tipos de canales por los que se originó el evento existen tanto nulos como 'Unknown'. Esto es un poco extraño dado que ambos parecen aportar la misma información, es decir, que no se sabe cuál es el canal por el que se originó el evento. Para ver porqué ocurre esto investigemos un poco más que otra información hay en los eventos de tipo 'Unknown':

In [36]:
df[df['channel'] == 'Unknown'].shape

(9, 23)

Notamos que son solo 9 registros, por lo que podemos analizarlos directamente:

In [37]:
df[df['channel'] == 'Unknown']

,timestamp,event,person,url,sku,model,condition,storage,color,skus,search_term,staticpage,campaign_source,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
397774,2018-04-30 20:21:46,visited site,689eb127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,Returning,Rio de Janeiro,Rio de Janeiro,Brazil,Computer,1366x768,Windows 7,Firefox 45
397775,2018-05-02 10:48:47,visited site,689eb127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,Returning,Rio de Janeiro,Rio de Janeiro,Brazil,Computer,1366x768,Windows 7,Firefox 45
397776,2018-05-03 10:36:13,visited site,689eb127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,Returning,Rio de Janeiro,Rio de Janeiro,Brazil,Computer,1366x768,Windows 7,Firefox 45
397777,2018-05-03 16:49:38,visited site,689eb127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,Returning,Rio de Janeiro,Rio de Janeiro,Brazil,Computer,1366x768,Windows 7,Firefox 45
397778,2018-05-04 17:03:44,visited site,689eb127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,Returning,Rio de Janeiro,Rio de Janeiro,Brazil,Computer,1366x768,Windows 7,Firefox 45
397779,2018-05-07 11:11:54,visited site,689eb127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,Returning,Rio de Janeiro,Rio de Janeiro,Brazil,Computer,1366x768,Windows 7,Firefox 45
397780,2018-05-07 20:24:34,visited site,689eb127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,Returning,Rio de Janeiro,Rio de Janeiro,Brazil,Computer,1366x768,Windows 7,Firefox 45
397781,2018-05-08 18:02:50,visited site,689eb127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,Returning,Rio de Janeiro,Rio de Janeiro,Brazil,Computer,1366x768,Windows 7,Firefox 45
574380,2018-05-31 23:41:10,visited site,92f05ad6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,New,Rio de Janeiro,Rio de Janeiro,Brazil,Computer,1360x768,Windows 7,Firefox 45


De esta forma nos damos cuenta que frente a la totalidad de registros no es un número representativo y podemos cambiarlos por  nulos (o también los nulos por 'Unknown') sin problemas.
Como este escenario lo encontramos en varios categorias realizamos este cambio con todas ellas.

In [38]:
for categoria in atributos_categoricos:
    df[categoria].loc[df[categoria] == 'Unknown'] = np.nan
    df[categoria].cat.remove_unused_categories(inplace=True)

<h4 id="new_vs_returning">new_vs_returning</h4>

In [39]:
df['new_vs_returning'].cat.categories

Index(['New', 'Returning'], dtype='object')

<h4 id="city">city</h4>

In [40]:
df['city'].cat.categories

Index(['Abadiania', 'Abaete', 'Abaetetuba', 'Abrantes', 'Abreu e Lima',
       'Acailandia', 'Acajutiba', 'Acarau', 'Acegua', 'Acu',
       ...
       'Várzea Grande', 'Wagner', 'Wanderley', 'Wrexham', 'Xambioa', 'Xanxere',
       'Xavantina', 'Xexeu', 'Zdunska Wola', 'Ze Doca'],
      dtype='object', length=1938)

<h4 id="region">region</h4>

In [41]:
df['region'].cat.categories

Index(['Acre', 'Aichi', 'Alagoas', 'Amapa', 'Amazonas', 'Arkansas', 'Asuncion',
       'Bahia', 'Basel-City', 'British Columbia', 'Buenos Aires',
       'Buenos Aires F.D.', 'California', 'Castelo Branco', 'Ceara',
       'Cidade de Maputo', 'Colorado', 'Connecticut', 'Cordoba', 'Delaware',
       'Departamento de Montevideo', 'Dhaka', 'England', 'Espirito Santo',
       'Essonne', 'Federal District', 'Florida', 'Gauteng', 'Georgia', 'Goias',
       'Hesse', 'Illinois', 'Indiana', 'Inner Mongolia Autonomous Region',
       'Iowa', 'Istanbul', 'Judetul Bacau', 'Kenitra Province', 'Loiret',
       'Lorraine', 'Louisiana', 'Maharashtra', 'Maranhao', 'Mato Grosso',
       'Mato Grosso do Sul', 'Mendoza', 'Mexico City', 'Michigan', 'Milan',
       'Minas Gerais', 'Minnesota', 'Missouri', 'Monagas', 'Moscow',
       'Nacional', 'Neuquen', 'Nevada', 'New Jersey', 'New York',
       'North Holland', 'Ontario', 'Para', 'Parana', 'Paraíba', 'Paris',
       'Pennsylvania', 'Pernambuco', 'Piaui', 

<h4 id="country">country</h4>

In [42]:
df['country'].cat.categories

Index(['Algeria', 'Angola', 'Argentina', 'Bangladesh', 'Belize', 'Brazil',
       'Canada', 'Cape Verde', 'China', 'Colombia', 'Denmark',
       'Dominican Republic', 'France', 'French Guiana', 'Germany',
       'Guadeloupe', 'India', 'Ireland', 'Israel', 'Italy', 'Japan', 'Mexico',
       'Morocco', 'Mozambique', 'Netherlands', 'Paraguay', 'Peru',
       'Philippines', 'Poland', 'Portugal', 'Romania', 'Russia', 'Rwanda',
       'Saudi Arabia', 'South Africa', 'Spain', 'Sri Lanka', 'Switzerland',
       'Turkey', 'United Kingdom', 'United States', 'Uruguay', 'Uzbekistan',
       'Venezuela', 'Zimbabwe'],
      dtype='object')

<h4 id="device_type">device_type</h4>

In [43]:
df['device_type'].cat.categories

Index(['Computer', 'Smartphone', 'Tablet'], dtype='object')

<h4 id="operating_system_version">operating_system_version</h4>

In [44]:
df['operating_system_version'].cat.categories

Index(['Android ', 'Android 2.3.6', 'Android 3.1', 'Android 3.2',
       'Android 4.0.3', 'Android 4.0.4', 'Android 4.1.1', 'Android 4.1.2',
       'Android 4.2.2', 'Android 4.3',
       ...
       'iOS 8.1', 'iOS 8.1.1', 'iOS 8.1.3', 'iOS 8.3', 'iOS 8.4', 'iOS 9.1',
       'iOS 9.2.1', 'iOS 9.3.2', 'iOS 9.3.4', 'iOS 9.3.5'],
      dtype='object', length=121)

<h4 id="browser_version">browser_version</h4>

In [45]:
df['browser_version'].cat.categories

Index(['Amazon Silk 66.3', 'Android 2.3', 'Android 3.1', 'Android 3.2',
       'Android 4.0', 'Android 4.1', 'Android 4.2', 'Android 4.3',
       'Android 4.4', 'Android 5.1',
       ...
       'UC Browser 12', 'UC Browser 12.2', 'UC Browser 12.5', 'UC Browser 7.0',
       'Vivaldi 1.94', 'Vivaldi 1.95', 'Vivaldi 1.96', 'Yandex Browser 16.9',
       'Yandex Browser 18.4', 'Yandex Browser 18.6'],
      dtype='object', length=343)

<h2 id="otros-atributos">Otros atributos</h2>

<h4 id="screen_resolution">screen_resolution</h4>

Este atributo es un poco particular, dado que resulta difícil clasificarlo. Esto se debe a que la resolución de una pantalla podemos pensarla como un atributo categórico, dado que la cantidad de resoluciones es limitada o como uno numérico (el resultado del ancho por el alto). Sin embargo ambas tienen desventajas: si lo consideramos categórico perdemos, en parte, la manera de realizar consultas del tipo 'resulciones más chicas que' de manera sencilla y si consideramos el ancho por el alto perdemos información sobre las dimensiones de los dispositivos.
Es por eso que se decidió dividir este atributo en dos: screen_resolution_with (ancho) y screen_resolution_height (alto) siendo ambos numéricos.

In [46]:
resolutions = df['screen_resolution'].str.split('x', expand=True)

In [47]:
df['screen_resolution_width'] = pd.to_numeric(resolutions[0])
df['screen_resolution_height'] =pd.to_numeric(resolutions[1])
df.drop(columns=['screen_resolution']);

## Devuelvo el dataframe
Para reutilizar el dataframe limpio en otras notebooks creo una función que lo devuelva.

In [50]:
def get_clean_df():
    return df